In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
import copy

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [3]:
mean = np.array([0.485,0.456,0.406])
std = np.array([0.229, 0.224, 0.225])

In [4]:
data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize(mean, std)
    ]),
    'val': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize(mean, std)
    ])
}

In [5]:
data_dir = './hymenoptera_data'
image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir,x), data_transforms[x]) for x in ['train', 'val']}
data_loaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=4, shuffle=True) for x in ['train', 'val']}
dataset_sizes = {x: len(image_datasets[x]) for x in ['train','val']}
class_names = image_datasets['train'].classes
class_names

['ants', 'bees']

In [12]:
def train_model(model, criterion, optimizer, scheduler, n_epochs=25):
    since = time.time()
    
    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0
    
    for epoch in range(n_epochs):
        print(f'Epoch {epoch}/{n_epochs-1}')
        print('-'*10)
        
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()
            else:
                model.eval()
    
            running_loss = 0.0
            running_corrects = 0
            
            for inputs, labels in data_loaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)
                
                # forward pass
                # track grad only if in train
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)
                    
                    # backwards pass + optimize if in train
                    if phase == 'train':
                        optimizer.zero_grad()
                        loss.backward()
                        optimizer.step()
                
                running_loss += loss.item()*inputs.size(0)
                running_corrects += torch.sum(preds==labels.data)
                
            if phase == 'train':
                scheduler.step()
            
            epoch_loss = running_loss/dataset_sizes[phase]
            epoch_acc = running_corrects/dataset_sizes[phase]
            
            print(f'{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')
            
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                # save model weights
                best_model_wts = copy.deepcopy(model.state_dict())
                
        print()
        
    time_elapsed = time.time() - since
    
    print(f'Training completed in {time_elapsed//60:.0f}m {time_elapsed%60:.0f}')
    print(f'Best acc {best_acc:.4f}')
    
          
    # load back best model weights
    model.load_state_dict(best_model_wts)
    return model

In [13]:
model = models.resnet18(pretrained=True) # import model
num_features = model.fc.in_features # getting input features in last layer

model.fc = nn.Linear(num_features, 2) # changing final layer to output 2 classes

model.to(device)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [14]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001)

In [15]:
# using a scheduler to update learning rate

step_lr_scheduler = lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1) # every 7 epochs, lr is multiplied by gamma

In [16]:
model = train_model(model, criterion, optimizer, step_lr_scheduler, n_epochs=5)

Epoch 0/4
----------
train Loss: 0.6296 Acc: 0.6680
val Loss: 0.4194 Acc: 0.8693

Epoch 1/4
----------
train Loss: 0.5203 Acc: 0.7869
val Loss: 0.3541 Acc: 0.9020

Epoch 2/4
----------
train Loss: 0.4390 Acc: 0.8033
val Loss: 0.2693 Acc: 0.9216

Epoch 3/4
----------
train Loss: 0.4794 Acc: 0.7541
val Loss: 0.2430 Acc: 0.9281

Epoch 4/4
----------
train Loss: 0.3611 Acc: 0.8443
val Loss: 0.2193 Acc: 0.9281

Training completed in 1m 52
Best acc 0.9281


In [ ]:
# second option - only train last layer, freeze earlier layers

model = models.resnet18(pretrained=True) # import model

for param in model.parameters():
    param.requires_grad = False

num_features = model.fc.in_features # getting input features in last layer

model.fc = nn.Linear(num_features, 2) # changing final layer to output 2 classes

model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001)

step_lr_scheduler = lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1) # every 7 epochs, lr is multiplied by gamma

model = train_model(model, criterion, optimizer, step_lr_scheduler, n_epochs=5)

Epoch 0/4
----------
